# Etapa 0: Importar e ler dados

In [ ]:
# Importando bibliotecas e dataset
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns

plt.style.use('ggplot')
# pd.set_option('max_columns', 200)


In [ ]:
df = pd.read_csv('defeitos_dados.csv')

In [ ]:
df.head()

# Etapa 1: Compreensão dos Dados

* Formato do dataframe
* Cabeçalho e cauda
* Tipos de dados
* Descrição


In [ ]:
# Exibe a quantidade de linhas e colunas
df.shape

In [ ]:
# Exibe as 5 primeiras colunas
df.head()

In [ ]:
# Exibe o nome das colunas
df.columns

In [ ]:
# Seleciona apenas as colunas desejadas e cria uma cópia independente do DataFrame

df_filt = df[[
    'data_defeito',
    'local_defeito',
    'severidade',
    'metodo_inspecao',
    'custo_reparo'
]].copy()


In [ ]:
# Exemplo de remoção de colunas
df.drop(['local_defeito'], axis=1)

In [ ]:
#Exibe as colunas e seu tipo de dados
df.dtypes

In [ ]:
# Gera um resumo estatístico das colunas numéricas do DataFrame 'df'.
# Mostra informações como média, desvio padrão, mínimo, máximo e quartis.
# Útil para entender rapidamente a distribuição dos dados.
df.describe()

# Step 2: Preparação dos Dados

* Remoção de colunas e linhas irrelevantes
* Identificação de colunas duplicadas
* Renomeação de colunas
* Criação de variáveis (features)


In [ ]:
# Verifica se existem valores nulos (NaN) em cada coluna do DataFrame 'df_filt'.
# O método .isna() retorna True para cada célula que é nula.
# O método .sum() soma esses valores True (contados como 1),
# resultando no total de valores nulos por coluna.
df_filt.isna().sum()

In [ ]:
# Retorna todas as linhas duplicadas do DataFrame 'df_filt',
# exceto a primeira ocorrência de cada grupo de duplicatas.
# O método .duplicated() marca como True todas as linhas que já apareceram antes,
# considerando por padrão todas as colunas.
# O .loc[...] seleciona apenas essas linhas duplicadas.
df_filt.loc[df_filt.duplicated()]

In [ ]:
# Mostra quantas vezes cada valor aparece em uma coluna do DataFrame.
# Basta substituir 'metodo_inspecao' pelo nome da coluna que você deseja analisar.
# Útil para identificar a frequência de categorias, textos ou números em qualquer conjunto de dados.
df_filt['metodo_inspecao'].value_counts()

In [ ]:
# Seleciona todas as linhas do DataFrame 'df_filt' que possuem valores duplicados
# na coluna 'local_defeito'. Útil para identificar registros repetidos de defeitos
# no mesmo local.
df_filt.loc[df_filt.duplicated(subset=['metodo_inspecao'])]

In [ ]:
# Filtra o DataFrame 'df_filt' para mostrar apenas as linhas em que
# a coluna 'metodo_inspecao' possui o valor "Visual Inspection".
# Útil para analisar registros que foram inspecionados especificamente
# pelo método de inspeção visual.
df_filt.query('metodo_inspecao == "Visual Inspection"')

In [ ]:
# Filtra o DataFrame 'df_filt' para mostrar apenas as linhas em que:
# - a coluna 'metodo_inspecao' possui o valor "Visual Inspection"
# - e a coluna 'local_defeito' possui o valor "Internal".
# Útil para analisar registros que atendem a duas condições ao mesmo tempo.
df_filt.query('metodo_inspecao == "Visual Inspection" & local_defeito == "Internal"')

In [ ]:
# Mostra todas as colunas existentes no DataFrame 'df_filt'.
# Útil para verificar os nomes das colunas disponíveis antes de fazer filtros,
# contagens ou análises específicas.
df_filt.columns

In [ ]:
# Conta quantas linhas do DataFrame 'df_filt' possuem valores duplicados
# considerando ao mesmo tempo as colunas 'data_defeito' e 'severidade'.
# O parâmetro 'subset' deixa explícito quais colunas devem ser analisadas.
# O método .sum() retorna o total dessas duplicatas.
# Se usar "~" antes do comando ele da o inverso, somente aquilo que não se repete.
df_filt.duplicated(subset=['data_defeito','severidade']).sum()

In [ ]:
# Seleciona todas as linhas únicas do DataFrame 'df_filt',
# ou seja, aquelas que NÃO se repetem considerando ao mesmo tempo
# as colunas 'data_defeito', 'local_defeito', 'severidade' e 'metodo_inspecao'.
# O operador "~" inverte o resultado de .duplicated(),
# retornando apenas os registros não duplicados (primeira ocorrência de cada combinação).
df_filt.loc[~df_filt.duplicated(subset=['data_defeito','local_defeito','severidade','metodo_inspecao'])]

In [ ]:
# Cria um novo DataFrame chamado 'df_unico' contendo apenas as linhas únicas de 'df_filt',
# ou seja, aquelas que NÃO se repetem considerando as colunas
# 'data_defeito', 'local_defeito', 'severidade' e 'metodo_inspecao'.
# O método .reset_index(drop=True) redefine os índices para uma nova sequência (0, 1, 2, ...).
df_unico = df_filt.loc[~df_filt.duplicated(subset=['data_defeito','local_defeito','severidade','metodo_inspecao'])] \
    .reset_index(drop=True)

In [ ]:
# Mostra o número de linhas e colunas do DataFrame 'df_unico'.
# O resultado é uma tupla no formato (linhas, colunas).
# Útil para verificar rapidamente o tamanho do DataFrame após remover duplicatas.
df_unico.shape

# Step 3: Compreensão das Características

* Plotagem das Distribuições das Características
    * Histograma
    * KDE
    * Boxplot


In [ ]:
df_unico

In [ ]:
# Converte a coluna 'data_defeito' para o formato datetime (caso ainda não esteja).
df_unico['data_defeito'] = pd.to_datetime(df_unico['data_defeito'], dayfirst=True)

# Cria uma nova coluna chamada 'mes_defeito' contendo apenas o mês e ano da data.
# O formato '%Y-%m' gera valores como '2024-01', '2024-02', etc.
df_unico['mes_defeito'] = df_unico['data_defeito'].dt.to_period('M').astype(str)

# Conta quantos registros existem por mês.
df_unico['mes_defeito'].value_counts().sort_index()

In [ ]:
# Conta os defeitos por mês e ordena pelo índice (que são os meses).
# Em seguida plota o gráfico de linha com os meses em ordem cronológica.
grafico_linha = df_unico['mes_defeito'].value_counts().sort_index().plot(
    kind='line',
    title='Defeitos por mês')

grafico_linha.set_xlabel('Mês')
grafico_linha.set_ylabel('Defeitos')

In [ ]:
# Conta quantas vezes cada severidade aparece e plota em gráfico de barras
grafico_barras = df_unico['severidade'].value_counts().plot(
    kind='bar',
    title='Distribuição de Severidade'
)
grafico_barras.set_xlabel('Severidade')
grafico_barras.set_ylabel('Quantidade')

In [ ]:
df_unico.columns

In [ ]:
# Cria um histograma da coluna 'custo_reparo' do DataFrame 'df_unico'.
# O parâmetro 'bins=50' divide os valores em 50 intervalos,
# mostrando a distribuição dos custos de reparo.
# O título e os rótulos dos eixos ajudam a interpretar o gráfico.
grafico_histograma = df_unico['custo_reparo'].plot(
    kind='hist',
    bins=10,
    title='Distribuição de Custos'
)
grafico_histograma.set_xlabel('Custo')
grafico_histograma.set_ylabel('Frequência')

In [ ]:
# Cria um gráfico de densidade (KDE) da coluna 'custo_reparo' do DataFrame 'df_unico'.
# Enquanto o histograma mostra frequências absolutas (quantidade de ocorrências em cada faixa de custo),
# o KDE mostra uma curva suavizada que representa a probabilidade relativa de encontrar valores
# em determinadas faixas de custo. A área total sob a curva é igual a 1 (100%),
# ou seja, o eixo Y não mostra porcentagem nem quantidade direta, mas sim densidade de probabilidade.
# Picos mais altos indicam faixas de custo onde os reparos são mais comuns,
# enquanto regiões mais baixas indicam valores menos frequentes.
# O título e os rótulos dos eixos ajudam a interpretar o gráfico.
grafico_kde = df_unico['custo_reparo'].plot(
    kind='kde',
    title='Distribuição de Custos'
)
grafico_kde.set_xlabel('Custo')
grafico_kde.set_ylabel('Densidade')

# Step 4: Relações entre Características

* Diagrama de Dispersão
* Mapa de Calor de Correlação
* Diagrama de Pares
* Comparações por Agrupamento